# Build a Dashboard Application with Plotly Dash

In [ ]:
#pip install pandas dash
#pip install wget
#import wget
#wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                 
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value='ALL',  # value attribute with default dropdown value to be ALL meaning all sites are selected
                                             placeholder='Select a Launch Site here',  # placeholder attribute to show a text description about this input area
                                             searchable=True  # so we can enter keywords to search launch sites
                                            ),                  
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                               min=0, max=10000, step=1000,
                                               marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                               value=[min_payload,max_payload]
                                               ),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'),
             )
def get_pie_chart(entered_site):
    #if str(entered_site)=='All':
    if entered_site == 'ALL':
        data1=spacex_df[['Launch Site','class']].groupby('Launch Site').mean()
        data1.reset_index(inplace=True)
        data1.columns=['Launch Site', 'Everage Success Rate']
        fig1=px.pie(data1, values='Everage Success Rate', names="Launch Site", title="The Sucess Lauches by Site")
        # return dcc.Gragh(figure=fig1) 
        return fig1
    else:
        data2=spacex_df[spacex_df['Launch Site']== str(entered_site)]['class'].value_counts(normalize=True)
        data2=pd.DataFrame(data2)
        data2.reset_index(inplace=True)
        fig2=px.pie(data2, values='class', names='index', title=f'Total Success Launches for Site {entered_site}')
        #return dcc.Graph(figure=fig2)   
        return fig2
        

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')]
              )
def get_scatter_plot(entered_site, payload_range):
    low, high = payload_range
    #low=list(payload_range)[0].astype('float')
    #high=list(payload_range)[1].astype('float')
    data3=spacex_df[(spacex_df['Payload Mass (kg)']> low) & (spacex_df['Payload Mass (kg)'].astype('float')<high)]
    
    if entered_site == 'ALL':       
        fig3=px.scatter(data3, x='Payload Mass (kg)', y='class', title='Correlation between Payload and Success for all Sites', color="Booster Version Category")
        return fig3   
    else:
        #data4=spacex_df[spacex_df['Launch Site']==str(entered_site)][['Payload Mass (kg)', 'Booster Version Category', 'class']]
        data4=data3[data3['Launch Site']==str(entered_site)][['Payload Mass (kg)', 'Booster Version Category', 'class']]
        fig4=px.scatter(data4, x='Payload Mass (kg)', y='class', title=f'Correlation between Payload and Success for {entered_site}', color="Booster Version Category")
        return fig4
# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2021 18:06:29] "POST /_dash-update-component HTTP/1.1" 200 -
